<a href="https://colab.research.google.com/github/Computer-CGuy/etherpaisahipaisa/blob/master/JAX_%7C_SoilSample_%7C_NewEdition_BACKUP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import jax.tools.colab_tpu
jax.tools.colab_tpu.setup_tpu()

In [1]:
import jax
import numpy as np

In [2]:
FNV_PRIME = 0x01000193
def fnv(v1, v2):
    return ((v1 * FNV_PRIME) ^ v2)
fnv = jax.jit(fnv)

In [3]:
dts = 1024

In [4]:
# dataset = [np.random.randint(low=0,high=9999,size=(32,),dtype=np.uint32) for x in range(dts)]
dataset = np.random.randint(low=0,high=9999,size=(10,32),dtype=np.uint32)
dataset = jax.device_put(dataset)

In [ ]:
# Define factory
def factory():
    val = np.random.randint(low=0,high=9999,size=(32,),dtype=np.uint32)# for x in range(dts)
    def f(i):
        return (val)

    return f


# Define functions
functions = []
for i in range(dts):
    functions.append(factory())


In [ ]:
del functions

In [ ]:

# # Run functions
# for f in functions:
#     print(f())

In [ ]:
def sameOutput(input):
    return input+1

In [ ]:
# dataset[0]

In [ ]:
from jax import jit, partial
# @partial(jit, static_argnums=0)
def body_funct(mix_):
    print("Hello")
    for i in range(32):
        mix_=fnv(mix_,dataset[fnv(i ^ 5, mix_[i%32])%2])
        # mix_=fnv(mix_,fnv(i ^ 5, mix_[i%32]))
        # mix_=fnv(mix_,fnv(i ^ 5, jax.lax.switch(mix_[i%32],[sameOutput for x in range(dts)],dataset)))
        # mix_=fnv(mix_,fnv(i ^ 5, jax.lax.switch(mix_[i%32],functions,None)))
    return mix_

In [ ]:
jax.lax.switch(mix[0][2%32],functions,None)

DeviceArray([2854, 4514, 3710, 5323, 5237, 9163, 9187, 2112, 5803, 7953,
             8958, 9111, 9725, 4368, 4813, 4308,  952, 9081, 3947, 9405,
             2944, 4483, 1380, 9146, 9674,  293, 5515, 9836, 3041,  332,
             5999, 5862], dtype=uint32)

In [ ]:
1024*(1000*1000/0.154)/8

831168831.1688312

In [ ]:
batch=1000
batch2=1000
mix = np.random.randint(low=0,high=9999,size=(batch,batch2,32),dtype=np.uint32)
mix = jax.device_put(mix)

In [ ]:
jit(body_funct)(mix[0])

DeviceArray([4101531055, 2565478759, 1411712873, 1005986663, 3154534929,
             1895653744, 3559895607,  106195891, 2932254468, 2501234242,
             3850781580,  330534717, 2035509451, 3709836175, 3030990400,
             3334276614, 4085261276,  980643615,  968333692, 3969050371,
             3691357050, 1306284113, 2112877983, 1187819614, 3499552229,
             1388488217,  658980932, 2593091086, 2015442202, 4142910658,
             3882617270, 3110020333], dtype=uint32)

In [ ]:
%time jit(body_funct)(mix[0]).block_until_ready()

CPU times: user 13.2 ms, sys: 4.02 ms, total: 17.2 ms
Wall time: 22.5 ms


DeviceArray([ 646045629, 2923493008,  423935416, 4178238860, 1748312356,
             3300138746,  991332407, 3141193701,  483287090, 3146131667,
             2066739447,  748236919,  424203711, 2458535934,  635718315,
             2160377807,  911826670, 4180731793, 4170901715, 1820067702,
             3566208608, 4234940677,  314527668, 2830355724, 2232120675,
             4115329429,  995666291, 1558341295, 2750503844, 1090293412,
             1859480801, 4042203612], dtype=uint32)

In [ ]:
body_funct = jax.jit(body_funct)

In [ ]:
bdf = jit(jax.vmap(body_funct))

In [ ]:
%time bdf(mix).block_until_ready()

In [ ]:
from jax.experimental.maps import xmap,mesh

In [ ]:
bdf_x = jit(xmap(body_funct, in_axes=[0,1,...], out_axes=[0,1,...]))

/usr/local/lib/python3.7/dist-packages/jax/experimental/maps.py:418: UserWarning: xmap is an experimental feature and probably has bugs!
  warn("xmap is an experimental feature and probably has bugs!")


In [ ]:
%time _ = bdf_x(mix)

CPU times: user 19.9 ms, sys: 62.7 ms, total: 82.5 ms
Wall time: 152 ms


In [ ]:
devices = np.array(jax.devices()).reshape((2,4))

In [ ]:
%%time
with mesh(devices, ('x','y')):
    bdf_mesh = xmap(
        body_funct,
        # in_axes=({0: 'left', 1: 'right'}),
        in_axes=({0: 'left', 1: 'right'}),
        # out_axes=['left', 'right', ...],
        out_axes=['left', 'right', ...],
        axis_resources={'left': 'x', 'right': 'y'})(mix).block_until_ready()

CPU times: user 24.5 ms, sys: 80 ms, total: 104 ms
Wall time: 167 ms


/usr/local/lib/python3.7/dist-packages/jax/experimental/maps.py:418: UserWarning: xmap is an experimental feature and probably has bugs!
  warn("xmap is an experimental feature and probably has bugs!")


In [ ]:
with mesh(devices, ('x','y')):
    _ = bdf_mesh(mix)

Hello


ValueError: ignored

In [ ]:
%timeit -n 1 -r 1 jax.vmap(body_funct)(mix).block_until_ready()

1 loop, best of 1: 515 ms per loop
